# Immporte das bibliotecas e constantes

In [ ]:
!pip install --user mplhep
import mplhep as hep 

from google.colab import drive
drive.mount('/content/drive')

import h5py
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from joblib import load

PATH = '/content/drive/MyDrive/Amostras_4Anomalos_Macedo/'
PATH_ = '/content/drive/MyDrive/Amostras_4Anomalos_Macedo_Eventos/'
PATH2 = '/content/drive/MyDrive/LGBM_Multiclass_ANOMALO_WWCEP_ Drecision_Tree/'
raiz_s = 13000
plt.style.use([hep.style.ROOT, hep.style.firamath])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


# Abrindo os arquivos .h5
- Aplicando o corte na região de controle do Signal
- Corrigindo as colunas do Background


In [ ]:
# Corte na região de sinal e adicionado corte na Massa Invariante do W

def open_file_back( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset_columns = f['columns']
        dset_dados = f['dados']
        #print( '\n colunas --> ', np.array( dset_columns ),'\n' )
        df = pd.DataFrame( np.array(dset_dados), columns = np.array( dset_columns))
        df[b'Mpps'] = raiz_s * ( np.sqrt( df[b'xi1'] * df[b'xi2'] ) )
        df[b'Ypps'] = 1/2 * np.log( df[b'xi1'] / df[b'xi2'] )
        df[b'Mww/Mpps'] = df[b'W_Mass'] / df[b'Mpps']
        df[b'Ypps-Yww'] = df[b'Ypps'] - df[b'W_rapidity']
        df[b'label'] = 0

        #df_cut = (df[b'muon_pt'] > 53)  & (df[b'muon_eta'] < 2.4) & (df[b'jetAK8_pt'] > 200) & (df[b'btag'] == 0) &  (df[b'xi1'] > 0.04) & (df[b'xi2'] > 0.04) & (df[b'xi1'] < 0.111) & (df[b'xi2'] < 0.138)   & (df[b'jetAK8_prunedMass'] > 65) & (df[b'jetAK8_prunedMass'] < 105)  &  (df[b'jet_eta'] < 2.4) & (df[b'METPt'] > 40) & (df[b'W_pt_lep'] > 200)
        df_cut = (df[b'muon_pt'] > 53) & (df[b'xi1'] > 0.04) & (df[b'W_Mass'] > 600) & (df[b'xi2'] > 0.04) & (df[b'xi1'] < 0.111) & (df[b'xi2'] < 0.138) & (df[b'muon_eta'] < 2.4) & (df[b'jetAK8_pt'] > 200) & (df[b'jet_eta'] < 2.4) & (df[b'METPt'] > 40) & (df[b'W_pt_lep'] > 200)

        dset = df[df_cut]
        return dset


def open_file_dados( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset_columns = f['columns']
        dset_dados = f['dados']
        #print( '\n colunas --> ', np.array( dset_columns ),'\n' )
        df = pd.DataFrame( np.array(dset_dados), columns = np.array( dset_columns))
        df[b'Mpps'] = raiz_s * ( np.sqrt( df[b'xi1'] * df[b'xi2'] ) )
        df[b'Ypps'] = 1/2 * np.log( df[b'xi1'] / df[b'xi2'] )
        df[b'Mww/Mpps'] = df[b'W_Mass'] / df[b'Mpps']
        df[b'Ypps-Yww'] = df[b'Ypps'] - df[b'W_rapidity']

        #df_cut = (df[b'muon_pt'] > 53)  & (df[b'muon_eta'] < 2.4) & (df[b'jetAK8_pt'] > 200) & (df[b'btag'] == 0) &  (df[b'xi1'] > 0.04) & (df[b'xi2'] > 0.04) & (df[b'xi1'] < 0.111) & (df[b'xi2'] < 0.138)   & (df[b'jetAK8_prunedMass'] > 65) & (df[b'jetAK8_prunedMass'] < 105)  &  (df[b'jet_eta'] < 2.4) & (df[b'METPt'] > 40) & (df[b'W_pt_lep'] > 200)
        df_cut = (df[b'muon_pt'] > 53) & (df[b'xi1'] > 0.04) & (df[b'xi2'] > 0.04) & (df[b'W_Mass'] > 600) & (df[b'xi1'] < 0.111) & (df[b'xi2'] < 0.138) & (df[b'muon_eta'] < 2.4) & (df[b'jetAK8_pt'] > 200) & (df[b'jet_eta'] < 2.4) & (df[b'METPt'] > 40) & (df[b'W_pt_lep'] > 200)
        dset = df[df_cut]
        return dset



def open_file_signal( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset_columns = f['columns']
        dset_dados = f['dados']
        #print( '\n colunas --> ', np.array( dset_columns ),'\n' )
        df = pd.DataFrame( np.array(dset_dados), columns = np.array( dset_columns))
        df[b'Mpps'] = raiz_s * ( np.sqrt( df[b'xi1'] * df[b'xi2'] ) )
        df[b'Ypps'] = 1/2 * np.log( df[b'xi1'] / df[b'xi2'] )
        df[b'Mww/Mpps'] = df[b'W_Mass'] / df[b'Mpps']
        df[b'Ypps-Yww'] = df[b'Ypps'] - df[b'W_rapidity']
        df[b'label'] = 1

        #df_cut = (df[b'muon_pt'] > 53)  & (df[b'muon_eta'] < 2.4) & (df[b'jetAK8_pt'] > 200) & (df[b'btag'] == 0) &  (df[b'xi1'] > 0.04) & (df[b'xi2'] > 0.04) & (df[b'xi1'] < 0.111) & (df[b'xi2'] < 0.138)   & (df[b'jetAK8_prunedMass'] > 65) & (df[b'jetAK8_prunedMass'] < 105)  &  (df[b'jetAK8_eta'] < 2.4) & (df[b'METPt'] > 40) & (df[b'W_pt_lep'] > 200)
        df_cut = (df[b'muon_pt'] > 53) & (df[b'xi1'] > 0.04) & (df[b'xi2'] > 0.04) & (df[b'W_Mass'] > 600) & (df[b'xi1'] < 0.111) & (df[b'xi2'] < 0.138) & (df[b'muon_eta'] < 2.4) & (df[b'jetAK8_pt'] > 200) & (df[b'jetAK8_eta'] < 2.4) & (df[b'METPt'] > 40) & (df[b'W_pt_lep'] > 200)
        dset = df[df_cut]
        return dset






def open_file_SM( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset_columns = f['columns']
        dset_dados = f['dados']
        #print( '\n colunas --> ', np.array( dset_columns ),'\n' )
        df = pd.DataFrame( np.array(dset_dados), columns = np.array( dset_columns))
        df[b'Mpps'] = raiz_s * ( np.sqrt( df[b'xi1'] * df[b'xi2'] ) )
        df[b'Ypps'] = 1/2 * np.log( df[b'xi1'] / df[b'xi2'] )
        df[b'Mww/Mpps'] = df[b'W_Mass'] / df[b'Mpps']
        df[b'Ypps-Yww'] = df[b'Ypps'] - df[b'W_rapidity']
        df[b'label'] = 2

        #df_cut = (df[b'muon_pt'] > 53)  & (df[b'muon_eta'] < 2.4) & (df[b'jetAK8_pt'] > 200) & (df[b'btag'] == 0) &  (df[b'xi1'] > 0.04) & (df[b'xi2'] > 0.04) & (df[b'xi1'] < 0.111) & (df[b'xi2'] < 0.138)   & (df[b'jetAK8_prunedMass'] > 65) & (df[b'jetAK8_prunedMass'] < 105)  &  (df[b'jetAK8_eta'] < 2.4) & (df[b'METPt'] > 40) & (df[b'W_pt_lep'] > 200)
        df_cut = (df[b'muon_pt'] > 53) & (df[b'xi1'] > 0.04) & (df[b'xi2'] > 0.04) & (df[b'W_Mass'] > 600) & (df[b'xi1'] < 0.111) & (df[b'xi2'] < 0.138) & (df[b'muon_eta'] < 2.4) & (df[b'jetAK8_pt'] > 200) & (df[b'jetAK8_eta'] < 2.4) & (df[b'METPt'] > 40) & (df[b'W_pt_lep'] > 200)
        dset = df[df_cut]
        return dset






In [ ]:
Drell_Yan = open_file_back(PATH_ + 'DataSet_multiRP_DrellYan.h5')
QCD = open_file_back(PATH_ + 'DataSet_multiRP_QCD.h5')
Single_top = open_file_back(PATH_ + 'DataSet_multiRP_single_top.h5')
VV_Inclusivo = open_file_back(PATH_ + 'DataSet_multiRP_VV_inclusivo.h5')
W_Jets = open_file_back(PATH_ + 'DataSet_multiRP_WJets.h5')
TT_bar = open_file_back(PATH_ + 'DataSet_TTbar.h5')

data = open_file_dados(PATH_ + 'DataSet_dados_multiRP.h5')

A01 = open_file_signal( PATH_ + 'output-DataSet_ANOMALO1_multiRP.h5' )
A02 = open_file_signal( PATH_ + 'output-DataSet_ANOMALO2_multiRP.h5' )
A03 = open_file_signal( PATH_ + 'output-DataSet_ANOMALO3_multiRP.h5' )
A04 = open_file_signal( PATH_ + 'output-DataSet_ANOMALO4_multiRP.h5' )
A05 = open_file_signal( PATH_ + 'output-DataSet_ANOMALO5_multiRP.h5' )
A06 = open_file_signal( PATH_ + 'output-DataSet_ANOMALO6_multiRP.h5' )
A07 = open_file_signal( PATH_ + 'output-DataSet_ANOMALO7_multiRP.h5' )
A08 = open_file_signal( PATH_ + 'output-DataSet_ANOMALO8_multiRP.h5' )

SM = open_file_SM( PATH_ + 'output-SM_multiRP.h5' )

In [ ]:
columns_ = [ b'W_Mass', b'W_pt_lep', b'dPhi_Whad_Wlep',b'dPhi_jatos_MET', b'jetAK8_pt', b'jetAK8_eta',
                b'jetAK8_prunedMass',      b'jetAK8_tau21',             b'METPt',
                b'muon_pt',          b'muon_eta',       b'ExtraTracks',
               b'PUWeight',        b'W_rapidity',              b'btag',
                b'xi1',               b'xi2',b'ismultirp1',        b'ismultirp2',
                b'Norm',
                  b'weight',              b'Mpps',              b'Ypps',
                b'Mww/Mpps',          b'Ypps-Yww', b'label']


columns_data = [ b'W_Mass', b'W_pt_lep', b'dPhi_Whad_Wlep',b'dPhi_jatos_MET', b'jetAK8_pt', b'jetAK8_eta',
                b'jetAK8_prunedMass',      b'jetAK8_tau21',             b'METPt',
                b'muon_pt',          b'muon_eta',       b'ExtraTracks',
               b'PUWeight',        b'W_rapidity',              b'btag',
                b'xi1',               b'xi2',b'ismultirp1',        b'ismultirp2',
                              b'Mpps',              b'Ypps',
                b'Mww/Mpps',          b'Ypps-Yww']




DrellYan = pd.DataFrame( np.array(Drell_Yan),columns=columns_ )
QCD_ = pd.DataFrame( np.array(QCD),columns=columns_ )
SingleTop = pd.DataFrame( np.array(Single_top),columns=columns_ )
VVInclusivo = pd.DataFrame( np.array(VV_Inclusivo),columns=columns_ )
Wjets = pd.DataFrame( np.array(W_Jets),columns=columns_ )
TTbar = pd.DataFrame( np.array(TT_bar),columns=columns_ )

data_set_dados_multirp = pd.DataFrame( np.array(data),columns=columns_data )

concat_back = pd.concat([DrellYan, QCD_, SingleTop, VVInclusivo, Wjets, TTbar])

concat_back[b'label'] = 0

# Preparando dados de teste e treino da Decision Tree

In [ ]:
DataSet_ANML01 = pd.concat([A01, concat_back, SM])
DataSet_ANML02 = pd.concat([A02, concat_back, SM])
DataSet_ANML03 = pd.concat([A03, concat_back, SM])
DataSet_ANML04 = pd.concat([A04, concat_back, SM])
DataSet_ANML05 = pd.concat([A05, concat_back, SM])
DataSet_ANML06 = pd.concat([A06, concat_back, SM])
DataSet_ANML07 = pd.concat([A07, concat_back, SM])
DataSet_ANML08 = pd.concat([A08, concat_back, SM])

In [ ]:
from sklearn.model_selection import train_test_split

test_size_ = 0.5

DataSet_Train1_, DataSet_Test1_ = train_test_split( DataSet_ANML01, test_size = test_size_, random_state=42, stratify=DataSet_ANML01[b'label'] )
DataSet_Train2_, DataSet_Test2_ = train_test_split( DataSet_ANML02, test_size = test_size_, random_state=42, stratify=DataSet_ANML02[b'label'] )
DataSet_Train3_, DataSet_Test3_ = train_test_split( DataSet_ANML03, test_size = test_size_, random_state=42, stratify=DataSet_ANML03[b'label'] )
DataSet_Train4_, DataSet_Test4_ = train_test_split( DataSet_ANML04, test_size = test_size_, random_state=42, stratify=DataSet_ANML04[b'label'] )
DataSet_Train5_, DataSet_Test5_ = train_test_split( DataSet_ANML05, test_size = test_size_, random_state=42, stratify=DataSet_ANML05[b'label'] )
DataSet_Train6_, DataSet_Test6_ = train_test_split( DataSet_ANML06, test_size = test_size_, random_state=42, stratify=DataSet_ANML06[b'label'] )
DataSet_Train7_, DataSet_Test7_ = train_test_split( DataSet_ANML07, test_size = test_size_, random_state=42, stratify=DataSet_ANML07[b'label'] )
DataSet_Train8_, DataSet_Test8_ = train_test_split( DataSet_ANML08, test_size = test_size_, random_state=42, stratify=DataSet_ANML08[b'label'] )

In [ ]:
label_train_ANML1 = DataSet_Train1_[b'label']
label_test_ANML1 = DataSet_Test1_[b'label']

label_train_ANML2 = DataSet_Train2_[b'label']
label_test_ANML2 = DataSet_Test2_[b'label']

label_train_ANML3 = DataSet_Train3_[b'label']
label_test_ANML3 = DataSet_Test3_[b'label']

label_train_ANML4 = DataSet_Train4_[b'label']
label_test_ANML4 = DataSet_Test4_[b'label']

label_train_ANML5 = DataSet_Train5_[b'label']
label_test_ANML5 = DataSet_Test5_[b'label']

label_train_ANML6 = DataSet_Train6_[b'label']
label_test_ANML6 = DataSet_Test6_[b'label']

label_train_ANML7 = DataSet_Train7_[b'label']
label_test_ANML7 = DataSet_Test7_[b'label']

label_train_ANML8 = DataSet_Train8_[b'label']
label_test_ANML8 = DataSet_Test8_[b'label']

In [ ]:
colunas = [b'W_Mass', b'W_pt_lep', b'dPhi_Whad_Wlep', b'dPhi_jatos_MET', b'jetAK8_pt', b'jetAK8_eta', b'jetAK8_prunedMass', b'jetAK8_tau21', b'METPt', b'muon_pt', b'muon_eta', b'ExtraTracks', b'W_rapidity', b'xi1', b'xi2',b'Mpps', b'Ypps', b'Mww/Mpps', b'Ypps-Yww']

In [ ]:
DataSet_Train1 = DataSet_Train1_[colunas]
DataSet_Test1 = DataSet_Test1_[colunas]

DataSet_Train2 = DataSet_Train2_[colunas]
DataSet_Test2 = DataSet_Test2_[colunas]

DataSet_Train3 = DataSet_Train3_[colunas]
DataSet_Test3 = DataSet_Test3_[colunas]

DataSet_Train4 = DataSet_Train4_[colunas]
DataSet_Test4 = DataSet_Test4_[colunas]

DataSet_Train5 = DataSet_Train5_[colunas]
DataSet_Test5 = DataSet_Test5_[colunas]

DataSet_Train6 = DataSet_Train6_[colunas]
DataSet_Test6 = DataSet_Test6_[colunas]

DataSet_Train7 = DataSet_Train7_[colunas]
DataSet_Test7 = DataSet_Test7_[colunas]

DataSet_Train8 = DataSet_Train8_[colunas]
DataSet_Test8 = DataSet_Test8_[colunas]

In [ ]:
DataSet_Test1_weight_backg = DataSet_Test1_[DataSet_Test1_[ b'label']==0][b'weight']
DataSet_Test1_weight_signal = DataSet_Test1_[DataSet_Test1_[ b'label']==1][b'weight']
DataSet_Test1_weight_SM = DataSet_Test1_[DataSet_Test1_[ b'label']==2][b'weight']
DataSet_Test1_weight = DataSet_Test1_[b'weight']


DataSet_Test8_weight_backg = DataSet_Test8_[DataSet_Test8_[ b'label']==0][b'weight']
DataSet_Test8_weight_signal = DataSet_Test8_[DataSet_Test8_[ b'label']==1][b'weight']
DataSet_Test8_weight_SM = DataSet_Test8_[DataSet_Test8_[ b'label']==2][b'weight']
DataSet_Test8_weight = DataSet_Test8_[b'weight']


DataSet_Test7_weight_backg = DataSet_Test7_[DataSet_Test7_[ b'label']==0][b'weight']
DataSet_Test7_weight_signal = DataSet_Test7_[DataSet_Test7_[ b'label']==1][b'weight']
DataSet_Test7_weight_SM = DataSet_Test7_[DataSet_Test7_[ b'label']==2][b'weight']
DataSet_Test7_weight = DataSet_Test7_[b'weight']


DataSet_Test6_weight_backg = DataSet_Test6_[DataSet_Test6_[ b'label']==0][b'weight']
DataSet_Test6_weight_signal = DataSet_Test6_[DataSet_Test6_[ b'label']==1][b'weight']
DataSet_Test6_weight_SM = DataSet_Test6_[DataSet_Test6_[ b'label']==2][b'weight']
DataSet_Test6_weight = DataSet_Test6_[b'weight']


DataSet_Test5_weight_backg = DataSet_Test5_[DataSet_Test5_[ b'label']==0][b'weight']
DataSet_Test5_weight_signal = DataSet_Test5_[DataSet_Test5_[ b'label']==1][b'weight']
DataSet_Test5_weight_SM = DataSet_Test5_[DataSet_Test5_[ b'label']==2][b'weight']
DataSet_Test5_weight = DataSet_Test5_[b'weight']


DataSet_Test2_weight_backg = DataSet_Test2_[DataSet_Test2_[ b'label']==0][b'weight']
DataSet_Test2_weight_signal = DataSet_Test2_[DataSet_Test2_[ b'label']==1][b'weight']
DataSet_Test2_weight_SM = DataSet_Test2_[DataSet_Test2_[ b'label']==2][b'weight']
DataSet_Test2_weight = DataSet_Test2_[b'weight']


DataSet_Test3_weight_backg = DataSet_Test3_[DataSet_Test3_[ b'label']==0][b'weight']
DataSet_Test3_weight_signal = DataSet_Test3_[DataSet_Test3_[ b'label']==1][b'weight']
DataSet_Test3_weight_SM = DataSet_Test3_[DataSet_Test3_[ b'label']==2][b'weight']
DataSet_Test3_weight = DataSet_Test3_[b'weight']


DataSet_Test4_weight_backg = DataSet_Test4_[DataSet_Test4_[ b'label']==0][b'weight']
DataSet_Test4_weight_signal = DataSet_Test4_[DataSet_Test4_[ b'label']==1][b'weight']
DataSet_Test4_weight_SM = DataSet_Test4_[DataSet_Test4_[ b'label']==2][b'weight']
DataSet_Test4_weight = DataSet_Test4_[b'weight']

# Abrindo arquivos já treinados - Voting Classifier

In [ ]:
search_result1_Multiclass_logloss_scale_pos_weight = load(PATH2 + 'VotingClassifier_Soft_Multiclass_ANOMALO1_DataDriven.joblib')
search_result2_Multiclass_logloss_scale_pos_weight = load(PATH2 + 'VotingClassifier_Soft_Multiclass_ANOMALO2_DataDriven.joblib')
search_result3_Multiclass_logloss_scale_pos_weight = load(PATH2 + 'VotingClassifier_Soft_Multiclass_ANOMALO3_DataDriven.joblib')
search_result4_Multiclass_logloss_scale_pos_weight = load(PATH2 + 'VotingClassifier_Soft_Multiclass_ANOMALO4_DataDriven.joblib')
search_result5_Multiclass_logloss_scale_pos_weight = load(PATH  + 'VotingClassifier_Soft_Multiclass_ANOMALO5**_DataDriven.joblib')
search_result6_Multiclass_logloss_scale_pos_weight = load(PATH2 + 'VotingClassifier_Soft_Multiclass_ANOMALO6_DataDriven.joblib')
search_result7_Multiclass_logloss_scale_pos_weight = load(PATH2 + 'VotingClassifier_Soft_Multiclass_ANOMALO7_DataDriven.joblib')
search_result8_Multiclass_logloss_scale_pos_weight = load(PATH2 + 'VotingClassifier_Soft_Multiclass_ANOMALO8_DataDriven.joblib')

# Distribuição de Probabilidade
- Predict

In [ ]:
from sklearn.metrics import make_scorer,fbeta_score,precision_score,recall_score,accuracy_score,log_loss,roc_auc_score,classification_report,f1_score,confusion_matrix,roc_curve,precision_recall_curve,average_precision_score

In [ ]:
predict1_VotingClassifier_multiclass_logloss_scale_pos_weight = search_result1_Multiclass_logloss_scale_pos_weight.predict(DataSet_Test1)
predict2_VotingClassifier_multiclass_logloss_scale_pos_weight = search_result2_Multiclass_logloss_scale_pos_weight.predict(DataSet_Test2)
predict3_VotingClassifier_multiclass_logloss_scale_pos_weight = search_result3_Multiclass_logloss_scale_pos_weight.predict(DataSet_Test3)
predict4_VotingClassifier_multiclass_logloss_scale_pos_weight = search_result4_Multiclass_logloss_scale_pos_weight.predict(DataSet_Test4)
predict5_VotingClassifier_multiclass_logloss_scale_pos_weight = search_result5_Multiclass_logloss_scale_pos_weight.predict(DataSet_Test5)
predict6_VotingClassifier_multiclass_logloss_scale_pos_weight = search_result6_Multiclass_logloss_scale_pos_weight.predict(DataSet_Test6)
predict7_VotingClassifier_multiclass_logloss_scale_pos_weight = search_result7_Multiclass_logloss_scale_pos_weight.predict(DataSet_Test7)
predict8_VotingClassifier_multiclass_logloss_scale_pos_weight = search_result8_Multiclass_logloss_scale_pos_weight.predict(DataSet_Test8)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bytes']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bytes']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bytes']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bytes']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.

# Contagem do número de eventos após o corte
- Usando o valor default

In [ ]:
predict_dados1 = search_result1_Multiclass_logloss_scale_pos_weight.predict_proba(data_set_dados_multirp[colunas])
predict_dados2 = search_result2_Multiclass_logloss_scale_pos_weight.predict_proba(data_set_dados_multirp[colunas])
predict_dados3 = search_result3_Multiclass_logloss_scale_pos_weight.predict_proba(data_set_dados_multirp[colunas])
predict_dados4 = search_result4_Multiclass_logloss_scale_pos_weight.predict_proba(data_set_dados_multirp[colunas])
predict_dados5 = search_result5_Multiclass_logloss_scale_pos_weight.predict_proba(data_set_dados_multirp[colunas])
predict_dados6 = search_result6_Multiclass_logloss_scale_pos_weight.predict_proba(data_set_dados_multirp[colunas])
predict_dados7 = search_result7_Multiclass_logloss_scale_pos_weight.predict_proba(data_set_dados_multirp[colunas])
predict_dados8 = search_result8_Multiclass_logloss_scale_pos_weight.predict_proba(data_set_dados_multirp[colunas])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bytes']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bytes']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bytes']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bytes']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.

# Usando o valor de corte como 0.5

In [ ]:
best_cut = 0.5

In [ ]:
n_eventos_backg_dps_corte1 = predict1_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML1 == 0 ] > best_cut
n_eventos_signal_dps_corte1 = predict1_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML1 == 1 ] > best_cut
n_eventos_SM_dps_corte1 = predict1_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML1 == 2 ] > best_cut


n_eventos_backg_dps_corte2 = predict2_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML2 == 0 ] > best_cut
n_eventos_signal_dps_corte2 = predict2_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML2 == 1 ] > best_cut
n_eventos_SM_dps_corte2 = predict2_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML2 == 2 ] > best_cut


n_eventos_backg_dps_corte3 = predict3_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML3 == 0 ] > best_cut
n_eventos_signal_dps_corte3 = predict3_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML3 == 1 ] > best_cut
n_eventos_SM_dps_corte3 = predict3_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML3 == 2 ] > best_cut


n_eventos_backg_dps_corte4 = predict4_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML4 == 0 ] > best_cut
n_eventos_signal_dps_corte4 = predict4_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML4 == 1 ] > best_cut
n_eventos_SM_dps_corte4 = predict4_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML4 == 2 ] > best_cut


n_eventos_backg_dps_corte5 = predict5_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML5 == 0 ] > best_cut
n_eventos_signal_dps_corte5 = predict5_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML5 == 1 ] > best_cut
n_eventos_SM_dps_corte5 = predict5_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML5 == 2 ] > best_cut


n_eventos_backg_dps_corte6 = predict6_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML6 == 0 ] > best_cut
n_eventos_signal_dps_corte6 = predict6_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML6 == 1 ] > best_cut
n_eventos_SM_dps_corte6 = predict6_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML6 == 2 ] > best_cut


n_eventos_backg_dps_corte7 = predict7_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML7 == 0 ] > best_cut
n_eventos_signal_dps_corte7 = predict7_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML7 == 1 ] > best_cut
n_eventos_SM_dps_corte7 = predict7_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML7 == 2 ] > best_cut


n_eventos_backg_dps_corte8 = predict8_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML8 == 0 ] > best_cut
n_eventos_signal_dps_corte8 = predict8_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML8 == 1 ] > best_cut
n_eventos_SM_dps_corte8 = predict8_VotingClassifier_multiclass_logloss_scale_pos_weight[ label_test_ANML8 == 2 ] > best_cut

In [ ]:
print('------------------ANOMALO1------------------')
print('Numero de eventos de background depois do corte -->', DataSet_Test1_weight_backg [ n_eventos_backg_dps_corte1 ].sum() / test_size_ )
print('Numero de eventos de signal depois do corte -->', DataSet_Test1_weight_signal[ n_eventos_signal_dps_corte1].sum() / test_size_ )
print('Numero de eventos de SM depois do corte -->', DataSet_Test1_weight_SM [ n_eventos_SM_dps_corte1 ].sum() / test_size_ )


------------------ANOMALO1------------------
Numero de eventos de background depois do corte --> 0.016949344637419136
Numero de eventos de signal depois do corte --> 4.208744505872212
Numero de eventos de SM depois do corte --> 0.05331533855542411


In [ ]:
print('------------------ANOMALO2------------------')
print('Numero de eventos de background depois do corte -->', DataSet_Test2_weight_backg [ n_eventos_backg_dps_corte2 ].sum() / test_size_ )
print('Numero de eventos de signal depois do corte -->', DataSet_Test2_weight_signal[ n_eventos_signal_dps_corte2].sum() / test_size_ )
print('Numero de eventos de SM depois do corte -->', DataSet_Test2_weight_SM [ n_eventos_SM_dps_corte2 ].sum() / test_size_ )


------------------ANOMALO2------------------
Numero de eventos de background depois do corte --> 0.0
Numero de eventos de signal depois do corte --> 0.07462531226336878
Numero de eventos de SM depois do corte --> 0.03576116965987043


In [ ]:
print('------------------ANOMALO3------------------')
print('Numero de eventos de background depois do corte -->', DataSet_Test3_weight_backg [ n_eventos_backg_dps_corte3 ].sum() / test_size_ )
print('Numero de eventos de signal depois do corte -->', DataSet_Test3_weight_signal[ n_eventos_signal_dps_corte3].sum() / test_size_ )
print('Numero de eventos de SM depois do corte -->', DataSet_Test3_weight_SM [ n_eventos_SM_dps_corte3 ].sum() / test_size_ )


------------------ANOMALO3------------------
Numero de eventos de background depois do corte --> 0.0016062964667151066
Numero de eventos de signal depois do corte --> 0.3031722830074729
Numero de eventos de SM depois do corte --> 0.04934104386148424


In [ ]:
print('------------------ANOMALO4------------------')
print('Numero de eventos de background depois do corte -->', DataSet_Test4_weight_backg [ n_eventos_backg_dps_corte4 ].sum() / test_size_ )
print('Numero de eventos de signal depois do corte -->', DataSet_Test4_weight_signal[ n_eventos_signal_dps_corte4].sum() / test_size_ )
print('Numero de eventos de SM depois do corte -->', DataSet_Test4_weight_SM [ n_eventos_SM_dps_corte4 ].sum() / test_size_ )


------------------ANOMALO4------------------
Numero de eventos de background depois do corte --> 0.0016062964667151066
Numero de eventos de signal depois do corte --> 0.7431258560669411
Numero de eventos de SM depois do corte --> 0.053414893299615


In [ ]:
print('------------------ANOMALO5------------------')
print('Numero de eventos de background depois do corte -->', DataSet_Test5_weight_backg [ n_eventos_backg_dps_corte5 ].sum() / test_size_ )
print('Numero de eventos de signal depois do corte -->', DataSet_Test5_weight_signal[ n_eventos_signal_dps_corte5].sum() / test_size_ )
print('Numero de eventos de SM depois do corte -->', DataSet_Test5_weight_SM [ n_eventos_SM_dps_corte5 ].sum() / test_size_ )


------------------ANOMALO5------------------
Numero de eventos de background depois do corte --> 0.0
Numero de eventos de signal depois do corte --> 0.055567128785089
Numero de eventos de SM depois do corte --> 0.03774942734183731


In [ ]:
print('------------------ANOMALO6------------------')
print('Numero de eventos de background depois do corte -->', DataSet_Test6_weight_backg [ n_eventos_backg_dps_corte6 ].sum() / test_size_ )
print('Numero de eventos de signal depois do corte -->', DataSet_Test6_weight_signal[ n_eventos_signal_dps_corte6].sum() / test_size_ )
print('Numero de eventos de SM depois do corte -->', DataSet_Test6_weight_SM [ n_eventos_SM_dps_corte6 ].sum() / test_size_ )


------------------ANOMALO6------------------
Numero de eventos de background depois do corte --> 0.0
Numero de eventos de signal depois do corte --> 0.194677178865226
Numero de eventos de SM depois do corte --> 0.04122099889378006


In [ ]:
print('------------------ANOMALO7------------------')
print('Numero de eventos de background depois do corte -->', DataSet_Test7_weight_backg [ n_eventos_backg_dps_corte7 ].sum() / test_size_ )
print('Numero de eventos de signal depois do corte -->', DataSet_Test7_weight_signal[ n_eventos_signal_dps_corte7].sum() / test_size_ )
print('Numero de eventos de SM depois do corte -->', DataSet_Test7_weight_SM [ n_eventos_SM_dps_corte7 ].sum() / test_size_ )


------------------ANOMALO7------------------
Numero de eventos de background depois do corte --> 0.010346905096279333
Numero de eventos de signal depois do corte --> 0.6432567381363425
Numero de eventos de SM depois do corte --> 0.04814450095894211


In [ ]:
print('------------------ANOMALO8------------------')
print('Numero de eventos de background depois do corte -->', DataSet_Test8_weight_backg [ n_eventos_backg_dps_corte8 ].sum() / test_size_ )
print('Numero de eventos de signal depois do corte -->', DataSet_Test8_weight_signal[ n_eventos_signal_dps_corte8].sum() / test_size_ )
print('Numero de eventos de SM depois do corte -->', DataSet_Test8_weight_SM [ n_eventos_SM_dps_corte8 ].sum() / test_size_ )


------------------ANOMALO8------------------
Numero de eventos de background depois do corte --> 0.042268173558140214
Numero de eventos de signal depois do corte --> 4.241444655801601
Numero de eventos de SM depois do corte --> 0.051546422434979276


# Erro associado a contagem de eventos

$\sigma = \sqrt{\sum_{i = 1}^{n} w_{i}^2}$

Somente para os anomalos 1 e 8

In [ ]:
from math import sqrt

In [ ]:
def erro_calculate(erro_topologia):
  passo1 = erro_topologia**2
  passo2 = passo1.sum()
  passo3 = sqrt(passo2)
  return passo3

In [ ]:
erro1_signal = DataSet_Test1_weight_signal[ n_eventos_signal_dps_corte1] / test_size_
erro1_backg = DataSet_Test1_weight_backg[n_eventos_backg_dps_corte1] / test_size_
erro1_SM = DataSet_Test1_weight_SM[n_eventos_SM_dps_corte1] / test_size_


erro8_signal = DataSet_Test8_weight_signal[n_eventos_signal_dps_corte8] / test_size_
erro8_backg = DataSet_Test8_weight_backg[n_eventos_backg_dps_corte8] / test_size_
erro8_SM = DataSet_Test8_weight_SM [ n_eventos_SM_dps_corte8 ].sum() / test_size_

In [ ]:
print('------------------ANOMALO1------------------')
print('\n')
print(f'Erro na região de Sinal do ANOMALO 1 --> {erro_calculate(erro1_signal)}')
print(f'Erro na região de Background do ANOMALO 1 -->  {erro_calculate(erro1_backg)}')
print(f'Erro na região do Modelo Padrão do ANOMALO 1 --> {erro_calculate(erro1_SM)}')
print('\n')
print('------------------ANOMALO8------------------')
print('\n')
print(f'Erro na região de Sinal do ANOMALO 8 --> {erro_calculate(erro8_signal)}')
print(f'Erro na região de Background do ANOMALO 8 --> {erro_calculate(erro8_backg)}')
print(f'Erro na região do Modelo Padrão do ANOMALO 8 --> {erro_calculate(erro8_SM)}')


------------------ANOMALO1------------------


Erro na região de Sinal do ANOMALO 1 --> 0.18410608953839322
Erro na região de Background do ANOMALO 1 -->  0.008840795958736904
Erro na região do Modelo Padrão do ANOMALO 1 --> 0.009975292403642


------------------ANOMALO8------------------


Erro na região de Sinal do ANOMALO 8 --> 0.1831080113395698
Erro na região de Background do ANOMALO 8 --> 0.01856896922290986
Erro na região do Modelo Padrão do ANOMALO 8 --> 0.051546422434979276
